### Парсер MOEX API (https://iss.moex.com/iss/reference/)


#### engines
https://iss.moex.com/iss/engines  
id (int32)	name (string:45)	title (string:765)  
1	stock	Фондовый рынок и рынок депозитов  
2	state	Рынок ГЦБ (размещение)  
3	currency	Валютный рынок  
4	futures	Срочный рынок  
5	commodity	Товарный рынок  
6	interventions	Товарные интервенции  
7	offboard	ОТС-система  
9	agro	Агро  
1012	otc	ОТС с ЦК  
1282	quotes	Квоты  
1326	money	Денежный рынок  


#### markets
https://iss.moex.com/iss/engines/currency/markets  
id (int32)	NAME (string:45)	title (string:765)  
1341	otcindices	Внебиржевые индексы  
10	selt	Биржевые сделки с ЦК  
34	futures	Поставочные фьючерсы  
41	index	Валютный фиксинг  
45	otc	Внебиржевой  

### get_moex_minute_candles

In [60]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_moex_minute_candles(pair, engine, date):
    """
    Получает минутные свечи для валютной пары на MOEX за указанную дату
    
    Параметры:
    currency_pair (str): Код валютной пары (например, 'USD/RUB', 'EUR/RUB')
    date (str): Дата в формате 'YYYY-MM-DD'
    
    Возвращает:
    DataFrame: Данные свечей
    """
    # Преобразуем валютную пару в формат MOEX
    # moex_pair = currency_pair.replace('/', '').upper()
    
    # Формируем URL для запроса
    if engine == 'currency':
        market = 'selt'
        board = 'CETS'
    else:
        market = 'forts'
        board = 'RFUD'
    url = f"https://iss.moex.com/iss/engines/{engine}/markets/{market}/boards/{board}/securities/{pair}/candles.json"
    
    # Параметры запроса
    params = {
        'from': date,
        'till': date,
        'interval': 1,  # 1 - минутный интервал
        'iss.meta': 'off'
    }
    
    try:
        # Отправляем GET-запрос
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        # Парсим JSON
        data = response.json()
        candles = data['candles']['data']
        
        # Преобразуем в DataFrame
        df = pd.DataFrame(candles)
        
        if not df.empty:
            # Переименовываем колонки
            df.columns = ['open', 'close', 'high', 'low', 'value', 'volume', 'begin', 'end']
            
            # Преобразуем timestamp в datetime
            df['begin'] = pd.to_datetime(df['begin'])
            df['end'] = pd.to_datetime(df['end'])
            
            # Сортируем по времени
            df = df.sort_values('begin')
            
            return df
        else:
            print(f"Нет данных для {pair} за {date}")
            return pd.DataFrame()
            
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе данных: {e}")
        return pd.DataFrame()

# Пример использования
if __name__ == "__main__":
    # Список валютных пар
    # currency_pairs = ['USD/RUB', 'EUR/RUB', 'CNY/RUB']
    currency_data = [
        ('currency', 'CNY000000TOD'), # CNYRUB_TOD,
        ('currency', 'CNYRUB_TOM'), # CNYRUB_TOM,
        ('currency', 'CNYRUBTODTOM'), # CNY_TODTOM,
        ('currency', 'CNYRUB_SPT'), # CNYRUB_SPT,
        ('currency', 'GLDRUBTODTOM'), # GLD_TODTOM,
        ('currency', 'GLDRUB_TOM'), # GLDRUB_TOM,
        ('currency', 'USD000000TOD'), # USDRUB_TOD,
        ('currency', 'USD000UTSTOM'), # USDRUB_TOM,
        ('currency', 'USDRUB_SPT'), # USDRUB_SPT,
        ('futures', 'USDRUBF'), # USDRUBF,
        ('futures', 'CNYRUBF'), # CNYRUBF,
        ('futures', 'GLDRUBF'), # GLDRUBF,
    ]
    
    # Дата (вчерашний день)
    # target_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
    target_date = '2024-10-31'
    
    # Собираем данные для всех пар
    all_data = {}
    
    for current_currency in currency_data:
        current_engine, currency_pair = current_currency
        print(f"Загружаем данные для {currency_pair} за {target_date}...")
        df = get_moex_minute_candles(currency_pair, current_engine, target_date)
        if not df.empty:
            df['engine'] = current_engine
            all_data[currency_pair] = df.copy()
            print(f"Получено {len(df)} свечей для {currency_pair}")
    
    # Пример сохранения в excel
    for pair, data in all_data.items():
        filename = f"moex_{pair.replace('/', '')}_{target_date}.xlsx"
        data.to_excel('C:\\Users\\22054689\\Documents\\moex_api_data\\' + filename, index=False, engine='openpyxl')
        print(f"Данные сохранены в {filename}")

Загружаем данные для CNY000000TOD за 2024-10-31...
Получено 465 свечей для CNY000000TOD
Загружаем данные для CNYRUB_TOM за 2024-10-31...
Получено 500 свечей для CNYRUB_TOM
Загружаем данные для CNYRUBTODTOM за 2024-10-31...
Получено 454 свечей для CNYRUBTODTOM
Загружаем данные для CNYRUB_SPT за 2024-10-31...
Нет данных для CNYRUB_SPT за 2024-10-31
Загружаем данные для GLDRUBTODTOM за 2024-10-31...
Получено 29 свечей для GLDRUBTODTOM
Загружаем данные для GLDRUB_TOM за 2024-10-31...
Получено 500 свечей для GLDRUB_TOM
Загружаем данные для USD000000TOD за 2024-10-31...
Нет данных для USD000000TOD за 2024-10-31
Загружаем данные для USD000UTSTOM за 2024-10-31...
Нет данных для USD000UTSTOM за 2024-10-31
Загружаем данные для USDRUB_SPT за 2024-10-31...
Нет данных для USDRUB_SPT за 2024-10-31
Загружаем данные для USDRUBF за 2024-10-31...
Получено 500 свечей для USDRUBF
Загружаем данные для CNYRUBF за 2024-10-31...
Получено 500 свечей для CNYRUBF
Загружаем данные для GLDRUBF за 2024-10-31...
Пол

In [61]:
all_df_list = []

# Пример сохранения в общий excel
for pair, data in all_data.items():
    data['ticker'] = pair
    all_df_list.append(data.copy())
    
    print(f"Данные добавлены в общий all_df {pair}")
    
all_df = pd.concat(all_df_list, ignore_index=True)

filename = f"moex_all_data_{target_date}.xlsx"
all_df.to_excel('C:\\Users\\22054689\\Documents\\moex_api_data\\' + filename, index=False, engine='openpyxl')

Данные добавлены в общий all_df CNY000000TOD
Данные добавлены в общий all_df CNYRUB_TOM
Данные добавлены в общий all_df CNYRUBTODTOM
Данные добавлены в общий all_df GLDRUBTODTOM
Данные добавлены в общий all_df GLDRUB_TOM
Данные добавлены в общий all_df USDRUBF
Данные добавлены в общий all_df CNYRUBF
Данные добавлены в общий all_df GLDRUBF


### get_moex_engine_turnovers_by_date

In [62]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_moex_engine_turnovers_by_date(engine, date):

    url = f"https://iss.moex.com/iss/engines/{engine}/turnovers.json"
    
    
    # Параметры запроса
    params = {
        'date': str(date.date()),
        'is_tonight_session': 'false',
        'iss.meta': 'off'
    }
    
    try:
        # Отправляем GET-запрос
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        # Парсим JSON
        data = response.json()
        turnovers = data['turnovers']['data']
        columns = data['turnovers']['columns']
        
        # Преобразуем в DataFrame
        df = pd.DataFrame(turnovers, columns=columns)
        
        if not df.empty:          
            return df
        else:
            print(f"Нет данных для {engine} за {date}")
            return pd.DataFrame()
            
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе данных: {e}")
        return pd.DataFrame()

# Пример использования
if __name__ == "__main__":
    engines = ['stock', 'currency', 'futures', 'TOTALS'
    ]
    
    # Собираем данные для всех пар
    all_df = []
    
    for _date in pd.date_range('2023-01-01', '2025-08-21'):
        for _engine in engines:
            print(f"Загружаем данные для {_engine} за {_date}...")
            df = get_moex_engine_turnovers_by_date(_engine, _date)
            if not df.empty:
                df['engine'] = _engine
                all_df.append(df.copy())
                print(f"Получено {len(df)} свечей для {_engine}")

#     # Пример сохранения в excel
#     for pair, data in all_data.items():
#         filename = f"moex_{pair.replace('/', '')}_{target_date}.xlsx"
#         data.to_excel('C:\\Users\\22054689\\Documents\\moex_api_data\\' + filename, index=False, engine='openpyxl')
#         print(f"Данные сохранены в {filename}")

Загружаем данные для stock за 2023-01-01 00:00:00...
Нет данных для stock за 2023-01-01 00:00:00
Загружаем данные для currency за 2023-01-01 00:00:00...
Нет данных для currency за 2023-01-01 00:00:00
Загружаем данные для futures за 2023-01-01 00:00:00...
Нет данных для futures за 2023-01-01 00:00:00
Загружаем данные для TOTALS за 2023-01-01 00:00:00...
Нет данных для TOTALS за 2023-01-01 00:00:00
Загружаем данные для stock за 2023-01-02 00:00:00...
Получено 17 свечей для stock
Загружаем данные для currency за 2023-01-02 00:00:00...
Получено 3 свечей для currency
Загружаем данные для futures за 2023-01-02 00:00:00...
Получено 3 свечей для futures
Загружаем данные для TOTALS за 2023-01-02 00:00:00...
Получено 1 свечей для TOTALS
Загружаем данные для stock за 2023-01-03 00:00:00...
Получено 17 свечей для stock


ZeroDivisionError: division by zero

In [26]:
real_all_df = pd.concat(all_df, ignore_index=True)

In [28]:
real_all_df.to_excel('get_moex_engine_turnovers.xlsx', index=False)

### get_moex_history_bonds_prices_by_date

In [25]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_moex_history_bonds_prices_by_date(date):


    url = f"https://iss.moex.com/iss/history/engines/stock/markets/bonds/sessions/1/securities.json"
    
    # Параметры запроса
    params = {
        'date': str(date.date()),
        'iss.meta': 'off',
        'start': 0
    }
    
    try:
        # Отправляем GET-запрос
        response = requests.get(url, params=params)
        response.raise_for_status()
                
        # Парсим JSON
        data = response.json()  
        
        cursor = pd.DataFrame(data["history.cursor"]['data'], columns=data["history.cursor"]['columns'])
        print(cursor["TOTAL"].iloc[0])
        
        _df_list = []
        for i in range((cursor["TOTAL"].iloc[0] // 100) + 1):
        
            params['start'] = i * 100
            response = requests.get(url, params=params)
            response.raise_for_status()
        
            # Парсим JSON
            data = response.json()
            turnovers = data['history']['data']
            columns = data['history']['columns']

            # Преобразуем в DataFrame
            df = pd.DataFrame(turnovers, columns=columns)
        
            if not df.empty:
                _df_list.append(df)
            else:
                pass
           
        if not _df_list:
            return pd.DataFrame()
        
        return pd.concat(_df_list, ignore_index=True)
            
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе данных: {e}")
        return pd.DataFrame()

# Пример использования
if __name__ == "__main__":    
    # Собираем данные для всех пар
    all_df = []
    
    for _date in pd.date_range('2024-12-01', '2024-12-31'):
        print(f"Загружаем данные за {_date}...")
        df = get_moex_history_bonds_prices_by_date(_date)
        if not df.empty:
            df['requests_date'] = _date
            all_df.append(df.copy())
            print(f"Получено {len(df)} цен для {_date}")

#     # Пример сохранения в excel
#     for pair, data in all_data.items():
#         filename = f"moex_{pair.replace('/', '')}_{target_date}.xlsx"
#         data.to_excel('C:\\Users\\22054689\\Documents\\moex_api_data\\' + filename, index=False, engine='openpyxl')
#         print(f"Данные сохранены в {filename}")

Загружаем данные за 2024-12-01 00:00:00...
0
Загружаем данные за 2024-12-02 00:00:00...
0
Загружаем данные за 2024-12-03 00:00:00...
0
Загружаем данные за 2024-12-04 00:00:00...
0
Загружаем данные за 2024-12-05 00:00:00...
0
Загружаем данные за 2024-12-06 00:00:00...
0
Загружаем данные за 2024-12-07 00:00:00...
0
Загружаем данные за 2024-12-08 00:00:00...
0
Загружаем данные за 2024-12-09 00:00:00...
0
Загружаем данные за 2024-12-10 00:00:00...
0
Загружаем данные за 2024-12-11 00:00:00...
0
Загружаем данные за 2024-12-12 00:00:00...
0
Загружаем данные за 2024-12-13 00:00:00...
0
Загружаем данные за 2024-12-14 00:00:00...
0
Загружаем данные за 2024-12-15 00:00:00...
0
Загружаем данные за 2024-12-16 00:00:00...
0
Загружаем данные за 2024-12-17 00:00:00...
0
Загружаем данные за 2024-12-18 00:00:00...
0
Загружаем данные за 2024-12-19 00:00:00...
0
Загружаем данные за 2024-12-20 00:00:00...
0
Загружаем данные за 2024-12-21 00:00:00...
0
Загружаем данные за 2024-12-22 00:00:00...
0
Загружаем 

In [2]:
real_all_df = pd.concat(all_df, ignore_index=True)

NameError: name 'pd' is not defined

In [24]:
real_all_df.to_excel('get_moex_history_bonds_prices_by_date_2024_2025q1.xlsx', index=False)

### get_moex_minute_candles_bonds

In [3]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_moex_minute_candles_bonds(pair, engine, date, end_date):
    """
    Получает минутные свечи для валютной пары на MOEX за указанную дату
    
    Параметры:
    currency_pair (str): Код валютной пары (например, 'USD/RUB', 'EUR/RUB')
    date (str): Дата в формате 'YYYY-MM-DD'
    
    Возвращает:
    DataFrame: Данные свечей
    """
    # Преобразуем валютную пару в формат MOEX
    # moex_pair = currency_pair.replace('/', '').upper()
    
    # Формируем URL для запроса
    if engine == 'currency':
        market = 'selt'
        board = 'CETS'
    elif engine == 'stock':
        market = 'bonds'
        board = None
    else:
        market = 'forts'
        board = 'RFUD'
        
    url = f"https://iss.moex.com/iss/engines/{engine}/markets/{market}/securities/{pair}/candles.json"    
    
    # Параметры запроса
    params = {
        'from': date,
        'till': end_date,
        'interval': 1,  # 1 - минутный интервал
        'iss.meta': 'off'
    }
    
    try:
        # Отправляем GET-запрос
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        # Парсим JSON
        data = response.json()
        candles = data['candles']['data']
        
        # Преобразуем в DataFrame
        df = pd.DataFrame(candles)
        
        if not df.empty:
            # Переименовываем колонки
            df.columns = ['open', 'close', 'high', 'low', 'value', 'volume', 'begin', 'end']
            
            # Преобразуем timestamp в datetime
            df['begin'] = pd.to_datetime(df['begin'])
            df['end'] = pd.to_datetime(df['end'])
            
            # Сортируем по времени
            df = df.sort_values('begin')
            
            return df
        else:
            print(f"Нет данных для {pair} за {date}")
            return pd.DataFrame()
            
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе данных: {e}")
        return pd.DataFrame()

    
if __name__ == "__main__":
    currency_data = [        
     ('stock', 'SU29022RMFS9'),
     ('stock', 'SU26241RMFS8'),
     ('stock', 'SU29023RMFS7'),
     ('stock', 'SU29026RMFS0'),
     ('stock', 'SU29019RMFS5'),
     ('stock', 'SU26244RMFS2'),
     ('stock', 'SU29015RMFS3'),
     ('stock', 'SU26243RMFS4'),
     ('stock', 'SU26240RMFS0'),
     ('stock', 'SU26238RMFS4'),
     ('stock', 'SU26239RMFS2'),
     ('stock', 'SU29013RMFS8'),
     ('stock', 'SU26232RMFS7'),
     ('stock', 'SU29018RMFS7'),
     ('stock', 'SU29017RMFS9'),
     ('stock', 'SU29017RMFS9'),
     ('stock', 'SU26242RMFS6'),
     ('stock', 'SU26226RMFS9'),
     ('stock', 'SU29021RMFS1'),
     ('stock', 'SU29027RMFS8'),
     ('stock', 'SU26237RMFS6'),
     ('stock', 'SU29024RMFS5'),
     ('stock', 'SU26219RMFS4'),
     ('stock', 'SU26229RMFS3'),
     ('stock', 'SU26224RMFS4'),
     ('stock', 'SU26228RMFS5'),
     ('stock', 'SU29009RMFS6'),
     ('stock', 'SU29014RMFS6'),
     ('stock', 'SU29025RMFS2'),
     ('stock', 'SU26207RMFS9'),
     ('stock', 'SU26235RMFS0'),
     ('stock', 'SU29006RMFS2'),
     ('stock', 'SU26218RMFS6'),
     ('stock', 'SU29008RMFS8'),
     ('stock', 'SU29010RMFS4'),
     ('stock', 'SU26212RMFS9'),
     ('stock', 'SU26221RMFS0'),
     ('stock', 'SU29020RMFS3'),
     ('stock', 'SU46020RMFS2'),
     ('stock', 'SU26236RMFS8'),
     ('stock', 'SU25085RMFS0'),
     ('stock', 'SU29007RMFS0'),
     ('stock', 'SU26225RMFS1'),
     ('stock', 'SU26230RMFS1'),
     ('stock', 'SU26233RMFS5'),
     ('stock', 'SU29006RMFS2')
    ]
    
    # Дата (вчерашний день)
    # target_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
    target_date = '2024-12-27'
    
    # Собираем данные для всех пар
    all_data = {}
    
    for current_currency in currency_data:
        current_engine, currency_pair = current_currency
        print(f"Загружаем данные для {currency_pair} за {target_date}...")
        df = get_moex_minute_candles_bonds(
            currency_pair,
            current_engine,
            target_date,
            end_date = (pd.to_datetime(target_date) + timedelta(days=1)).strftime('%Y-%m-%d')
        )
        if not df.empty:
            df['engine'] = current_engine
            all_data[currency_pair] = df.copy()
            print(f"Получено {len(df)} свечей для {currency_pair}")
    
#     # Пример сохранения в excel
#     for pair, data in all_data.items():
#         filename = f"moex_{pair.replace('/', '')}_{target_date}.xlsx"
#         data.to_excel('C:\\Users\\22054689\\Documents\\moex_api_data\\' + filename, index=False, engine='openpyxl')
#         print(f"Данные сохранены в {filename}")

Загружаем данные для SU29022RMFS9 за 2024-12-27...
Получено 48 свечей для SU29022RMFS9
Загружаем данные для SU26241RMFS8 за 2024-12-27...
Получено 500 свечей для SU26241RMFS8
Загружаем данные для SU29023RMFS7 за 2024-12-27...
Получено 151 свечей для SU29023RMFS7
Загружаем данные для SU29026RMFS0 за 2024-12-27...
Получено 185 свечей для SU29026RMFS0
Загружаем данные для SU29019RMFS5 за 2024-12-27...
Получено 102 свечей для SU29019RMFS5
Загружаем данные для SU26244RMFS2 за 2024-12-27...
Получено 500 свечей для SU26244RMFS2
Загружаем данные для SU29015RMFS3 за 2024-12-27...
Получено 183 свечей для SU29015RMFS3
Загружаем данные для SU26243RMFS4 за 2024-12-27...
Получено 500 свечей для SU26243RMFS4
Загружаем данные для SU26240RMFS0 за 2024-12-27...


KeyboardInterrupt: 

In [6]:
all_df_list = []

# Пример сохранения в общий excel
for pair, data in all_data.items():
    data['SECID'] = pair
    all_df_list.append(data.copy())
    
    print(f"Данные добавлены в общий all_df {pair}")
    
all_df = pd.concat(all_df_list, ignore_index=True)

filename = f"moex_all_data_{target_date}.xlsx"
all_df.to_excel('C:\\Users\\22054689\\Documents\\moex_api_data_bonds\\' + filename, index=False, engine='openpyxl')






Данные добавлены в общий all_df SU29022RMFS9
Данные добавлены в общий all_df SU26241RMFS8
Данные добавлены в общий all_df SU29023RMFS7
Данные добавлены в общий all_df SU29026RMFS0
Данные добавлены в общий all_df SU29019RMFS5
Данные добавлены в общий all_df SU26244RMFS2
Данные добавлены в общий all_df SU29015RMFS3
Данные добавлены в общий all_df SU26243RMFS4
Данные добавлены в общий all_df SU26240RMFS0
Данные добавлены в общий all_df SU26238RMFS4
Данные добавлены в общий all_df SU26239RMFS2
Данные добавлены в общий all_df SU29013RMFS8
Данные добавлены в общий all_df SU26232RMFS7
Данные добавлены в общий all_df SU29018RMFS7
Данные добавлены в общий all_df SU29017RMFS9
Данные добавлены в общий all_df SU26242RMFS6
Данные добавлены в общий all_df SU26226RMFS9
Данные добавлены в общий all_df SU29021RMFS1
Данные добавлены в общий all_df SU29027RMFS8
Данные добавлены в общий all_df SU26237RMFS6
Данные добавлены в общий all_df SU29024RMFS5
Данные добавлены в общий all_df SU26219RMFS4
Данные доб